<a href="https://colab.research.google.com/github/pocari-pixel/Modelling-and-Visualisation-Kelvin-Iria/blob/main/Modelling_and_Visualisation_Kelvin_Iria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving CreditCardDataFinal.csv to CreditCardDataFinal.csv


https://towardsdatascience.com/credit-card-fraud-detection-using-machine-learning-python-5b098d4a8edc

In [5]:
#Packages related to general operating system & warnings
import os 
import warnings
warnings.filterwarnings('ignore')
#Packages related to data importing, manipulation, exploratory data #analysis, data understanding
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import datetime as datetime
from termcolor import colored as cl # text customization
#Packages related to data visualizaiton
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#Setting plot sizes and type of plot
plt.rc("font", size=14)
plt.rcParams['axes.grid'] = True
plt.figure(figsize=(6,3))
plt.gray()
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import  PolynomialFeatures, KBinsDiscretizer, FunctionTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, OrdinalEncoder
import statsmodels.formula.api as smf
import statsmodels.tsa as tsa
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor, AdaBoostClassifier, AdaBoostRegressor 
from sklearn.svm import LinearSVC, LinearSVR, SVC, SVR
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

<Figure size 432x216 with 0 Axes>

In [84]:
data = pd.read_csv('CreditCardDataFinal.csv')
data.head()

,Transaction ID,Date,Day of Week,Time,Type of Card,Entry Mode,Amount,Type of Transaction,Merchant Group,Country of Transaction,Shipping Address,Country of Residence,Gender,Age,Bank,Fraud
0,#3577 209,14-Oct-20,Wednesday,19,Visa,Tap,£5,POS,Entertainment,United Kingdom,United Kingdom,United Kingdom,M,25.2,RBS,0
1,#3039 221,14-Oct-20,Wednesday,17,MasterCard,PIN,£288,POS,Services,USA,USA,USA,F,49.6,Lloyds,0
2,#2694 780,14-Oct-20,Wednesday,14,Visa,Tap,£5,POS,Restaurant,India,India,India,F,42.2,Barclays,0
3,#2640 960,13-Oct-20,Tuesday,14,Visa,Tap,£28,POS,Entertainment,United Kingdom,India,United Kingdom,F,51.0,Barclays,0
4,#2771 031,13-Oct-20,Tuesday,23,Visa,CVC,£91,Online,Electronics,USA,USA,United Kingdom,M,38.0,Halifax,1


In [50]:
Total_transactions = len(data)
normal = len(data[data.Fraud == 0])
fraudulent = len(data[data.Fraud == 1])
fraud_percentage = round(fraudulent/normal*100, 2)
print(cl('Total number of Trnsactions are {}'.format(Total_transactions), attrs = ['bold']))
print(cl('Number of Normal Transactions are {}'.format(normal), attrs = ['bold']))
print(cl('Number of fraudulent Transactions are {}'.format(fraudulent), attrs = ['bold']))
print(cl('Percentage of fraud Transactions is {}'.format(fraud_percentage), attrs = ['bold']))


Total number of Trnsactions are 100000
Number of Normal Transactions are 92805
Number of fraudulent Transactions are 7195
Percentage of fraud Transactions is 7.75


Data Preparation / Formatting

In [85]:
data['Transaction ID'] = data['Transaction ID'].str.replace('#','')
data['Transaction ID'] = data['Transaction ID'].str.replace(' ','')
data['Amount'] = data['Amount'].str.replace('£','')
data['Date']=pd.to_datetime(data['Date'])
# can use decide where to pu this --> 
# data['Age'].max()
# data['Age'] = data['Age'].astype('uint8', errors='ignore') #We'll used unsigned integer "uint8" with range 0 - 255 to save memory. Humans don't age past 255 years old!
data.head()

,Transaction ID,Date,Day of Week,Time,Type of Card,Entry Mode,Amount,Type of Transaction,Merchant Group,Country of Transaction,Shipping Address,Country of Residence,Gender,Age,Bank,Fraud
0,3577209,2020-10-14,Wednesday,19,Visa,Tap,5,POS,Entertainment,United Kingdom,United Kingdom,United Kingdom,M,25.2,RBS,0
1,3039221,2020-10-14,Wednesday,17,MasterCard,PIN,288,POS,Services,USA,USA,USA,F,49.6,Lloyds,0
2,2694780,2020-10-14,Wednesday,14,Visa,Tap,5,POS,Restaurant,India,India,India,F,42.2,Barclays,0
3,2640960,2020-10-13,Tuesday,14,Visa,Tap,28,POS,Entertainment,United Kingdom,India,United Kingdom,F,51.0,Barclays,0
4,2771031,2020-10-13,Tuesday,23,Visa,CVC,91,Online,Electronics,USA,USA,United Kingdom,M,38.0,Halifax,1


In [86]:
data=data.convert_dtypes()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Transaction ID          100000 non-null  string        
 1   Date                    100000 non-null  datetime64[ns]
 2   Day of Week             100000 non-null  string        
 3   Time                    100000 non-null  Int64         
 4   Type of Card            100000 non-null  string        
 5   Entry Mode              100000 non-null  string        
 6   Amount                  99994 non-null   string        
 7   Type of Transaction     100000 non-null  string        
 8   Merchant Group          99990 non-null   string        
 9   Country of Transaction  100000 non-null  string        
 10  Shipping Address        99995 non-null   string        
 11  Country of Residence    100000 non-null  string        
 12  Gender                  99996 n

In [10]:
data.head()

,Transaction ID,Date,Day of Week,Time,Type of Card,Entry Mode,Amount,Type of Transaction,Merchant Group,Country of Transaction,Shipping Address,Country of Residence,Gender,Age,Bank,Fraud
0,#3577 209,14-Oct-20,Wednesday,19,Visa,Tap,£5,POS,Entertainment,United Kingdom,United Kingdom,United Kingdom,M,25.2,RBS,0
1,#3039 221,14-Oct-20,Wednesday,17,MasterCard,PIN,£288,POS,Services,USA,USA,USA,F,49.6,Lloyds,0
2,#2694 780,14-Oct-20,Wednesday,14,Visa,Tap,£5,POS,Restaurant,India,India,India,F,42.2,Barclays,0
3,#2640 960,13-Oct-20,Tuesday,14,Visa,Tap,£28,POS,Entertainment,United Kingdom,India,United Kingdom,F,51.0,Barclays,0
4,#2771 031,13-Oct-20,Tuesday,23,Visa,CVC,£91,Online,Electronics,USA,USA,United Kingdom,M,38.0,Halifax,1


In [ ]:
data['Age'].max()

86.1

In [87]:
data['Age'] = data['Age'].astype('uint8', errors='ignore')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Transaction ID          100000 non-null  string        
 1   Date                    100000 non-null  datetime64[ns]
 2   Day of Week             100000 non-null  string        
 3   Time                    100000 non-null  Int64         
 4   Type of Card            100000 non-null  string        
 5   Entry Mode              100000 non-null  string        
 6   Amount                  99994 non-null   string        
 7   Type of Transaction     100000 non-null  string        
 8   Merchant Group          99990 non-null   string        
 9   Country of Transaction  100000 non-null  string        
 10  Shipping Address        99995 non-null   string        
 11  Country of Residence    100000 non-null  string        
 12  Gender                  99996 n

Dropping Missing Values

In [29]:
data.isnull().sum()

Transaction ID             0
Date                       0
Day of Week                0
Time                       0
Type of Card               0
Entry Mode                 0
Amount                     6
Type of Transaction        0
Merchant Group            10
Country of Transaction     0
Shipping Address           5
Country of Residence       0
Gender                     4
Age                        0
Bank                       0
Fraud                      0
dtype: int64

In [88]:
new_data = data.dropna(axis=0) #axis 0 will remove null values from row, 1 is for columns
print("Old data frame length:", len(data), "\nNew data frame length:", 
       len(new_data), "\nNumber of rows with at least 1 NA value: ",
       (len(data)-len(new_data)))

Old data frame length: 100000 
New data frame length: 99977 
Number of rows with at least 1 NA value:  23


In [89]:
data = new_data

In [68]:
data.shape

(99977, 16)

Dropping Duplicates

In [62]:
duplicateRowdata = data[data.duplicated()]
print(duplicateRowdata)

Empty DataFrame
Columns: [Transaction ID, Date, Day of Week, Time, Type of Card, Entry Mode, Amount, Type of Transaction, Merchant Group, Country of Transaction, Shipping Address, Country of Residence, Gender, Age, Bank, Fraud]
Index: []


No duplicated rows found as dataframe returned as empty.

In [44]:
print(pd.unique(data['Date']))
print(pd.unique(data['Day of Week']))
print(pd.unique(data['Time']))
print(pd.unique(data['Type of Card']))
print(pd.unique(data['Entry Mode']))
print(pd.unique(data['Amount']))
print(pd.unique(data['Type of Transaction']))
print(pd.unique(data['Merchant Group']))
print(pd.unique(data['Country of Transaction']))
print(pd.unique(data['Shipping Address']))
print(pd.unique(data['Country of Residence']))
print(pd.unique(data['Gender']))
print(pd.unique(data['Age']))
print(pd.unique(data['Bank']))
print(pd.unique(data['Fraud']))

['2020-10-14T00:00:00.000000000' '2020-10-13T00:00:00.000000000'
 '2020-10-16T00:00:00.000000000' '2020-10-15T00:00:00.000000000']
<StringArray>
['Wednesday', 'Tuesday', 'Thursday', 'Friday']
Length: 4, dtype: string
<IntegerArray>
[19, 17, 14, 23, 20, 18, 11,  1, 21,  0,  8,  9, 15, 22,  7,  3, 12,  4,  5,
 10, 13,  2, 16,  6, 24]
Length: 25, dtype: Int64
<StringArray>
['Visa', 'MasterCard']
Length: 2, dtype: string
<StringArray>
['Tap', 'PIN', 'CVC']
Length: 3, dtype: string
<StringArray>
[  '5', '288',  '28',  '91',  '30', '231', '154',  '39',  '17', '326',
 ...
  '68', '193', '232', '227',  '33', '295', '276', '205', '284', '278']
Length: 396, dtype: string
<StringArray>
['POS', 'Online', 'ATM']
Length: 3, dtype: string
<StringArray>
['Entertainment',      'Services',    'Restaurant',   'Electronics',
      'Children',       'Fashion',          'Food',      'Products',
  'Subscription',        'Gaming']
Length: 10, dtype: string
<StringArray>
['United Kingdom', 'USA', 'India', 'Rus

We can see from the output that the data is mostly cleaned apart from a few unique values in columns 'Date' and 'Bank'.

Starting with the 'Date' column we can see that we have transactions recorded on the 13th, 14th, 15th & 16th October 2020. However, we have been told that the transactions we're recorded on the 13th & 14th October 2020. To minimise errors, we will drop the columns with date recorded on the 15th & 16th October 2020.

In [94]:
date_data = data.drop(data.index[(data['Date'] == '2020-10-15') | (data['Date'] == '2020-10-16')])
print("Old data frame length:", len(data), "\nNew data frame length:", 
       len(date_data), "\nNumber of rows with dates 15th or 16th October 2020: ",
       (len(data)-len(date_data)))


Old data frame length: 99977 
New data frame length: 99975 
Number of rows with dates 15th or 16th October 2020:  2


In [95]:
data = date_data

In [96]:
data.shape

(99975, 16)

Secondly, we can see that in the 'Bank' column we can see that the bank some of the entries have the miss spelling of 'Barclays'. We will replace the entries having 'Bar**cays**' to 'Barclays'.

Key:

- **Transaction ID** <-- is our Index  
- Date: 
- Day of Week            
- Time                    
- Type of Card           
- Entry Mode               
- Amount                   
- Type of Transaction      
- Merchant Group           
- Country of Transaction  
- Shipping Address        
- Country of Residence    
- Gender                
- Age                    
- Bank                    
- Fraud 

In [ ]:
data.replace({'A': {0: 100, 4: 400}})

In [ ]:
data1 = data[data.isna().any(axis=1)]
data1.head()

SyntaxError: ignored

In [ ]:
data = data.dropna(axis=1) #axis 1 will remove null values from row, 0 is for columns
print(data.isnull().sum())

Transaction ID            0
Date                      0
Day of Week               0
Time                      0
Type of Card              0
Entry Mode                0
Type of Transaction       0
Country of Transaction    0
Country of Residence      0
Age                       0
Bank                      0
Fraud                     0
dtype: int64


In [ ]:
min(data.Amount), max(data.Amount)

TypeError: ignored

In [ ]:
df['Transaction ID'].is_unique

False

In [ ]:
df.dtypes

Transaction ID             object
Date                       object
Day of Week                object
Time                        int64
Type of Card               object
Entry Mode                 object
Amount                     object
Type of Transaction        object
Merchant Group             object
Country of Transaction     object
Shipping Address           object
Country of Residence       object
Gender                     object
Age                       float64
Bank                       object
Fraud                       int64
dtype: object

In [ ]:
df = df.astype({'Transaction ID': int})

In [ ]:
df.dtypes

Transaction ID              int64
Date                       object
Day of Week                object
Time                        int64
Type of Card               object
Entry Mode                 object
Amount                     object
Type of Transaction        object
Merchant Group             object
Country of Transaction     object
Shipping Address           object
Country of Residence       object
Gender                     object
Age                       float64
Bank                       object
Fraud                       int64
dtype: object

In [ ]:
df = df.set_index('Transaction ID', inplace=True)

In [ ]:
df.isnull().sum()

AttributeError: ignored

In [ ]:
df.get_dtype_counts()

AttributeError: ignored